In [21]:
import pandas as pd
import subprocess
import os
from pathlib import Path
import pysam
import numpy as np
import sys

In [22]:
wd_dir = '/beegfs/scratch/ric.broccoli/kubacki.michal/SRF_CUTandTAG/custom_pipeline/results'
os.chdir(wd_dir)

# Get the current working directory
current_dir = os.getcwd()

In [24]:
def ensure_bam_index(bam_file):
    """
    Ensure BAM file is indexed. Create index if it doesn't exist.
    
    Parameters:
    - bam_file: path to BAM file
    """
    if not os.path.exists(bam_file + '.bai'):
        print(f"Creating index for {bam_file}")
        try:
            pysam.index(bam_file)
        except Exception as e:
            print(f"Error creating index for {bam_file}: {str(e)}")
            sys.exit(1)

def calculate_coverage(bam_file, chrom, start, end):
    """
    Calculate mean coverage for a genomic region from a BAM file.
    
    Parameters:
    - bam_file: path to the BAM file
    - chrom: chromosome name
    - start: region start position (0-based)
    - end: region end position
    
    Returns:
    - float: mean coverage across the region
    """
    try:
        # Ensure BAM is indexed
        ensure_bam_index(bam_file)
        
        # Open BAM file
        with pysam.AlignmentFile(bam_file, "rb") as bam:
            # Get per-base coverage array for the region
            coverage_arrays = bam.count_coverage(chrom, start, end)
            
            # Sum all nucleotide counts per position
            total_coverage = np.sum(coverage_arrays, axis=0)
            
            # Calculate mean coverage across the region
            mean_coverage = np.mean(total_coverage)
            
            return mean_coverage
    except Exception as e:
        print(f"Warning: Error calculating coverage for {chrom}:{start}-{end} in {bam_file}")
        print(f"Error: {str(e)}")
        return 0

def get_sample_bams(tissue, condition):
    """
    Get list of BAM files for a specific tissue and condition.
    
    Parameters:
    - tissue: 'Neuron' or 'NSC'
    - condition: 'Endo' or 'Exo'
    
    Returns:
    - list: paths to BAM files for the specified condition
    """
    # Update these paths to match your actual BAM file locations
    bam_dict = {
        'Neuron': {
            'Endo': ["aligned/NeuM2.bam", "aligned/NeuM3.bam"],
            'Exo': ["aligned/NeuV1.bam", "aligned/NeuV2.bam", "aligned/NeuV3.bam"]
        },
        'NSC': {
            'Endo': ["aligned/NSCM1.bam", "aligned/NSCM2.bam", "aligned/NSCM3.bam"],
            'Exo': ["aligned/NSCv1.bam", "aligned/NSCv2.bam", "aligned/NSCv3.bam"]
        }
    }
    
    # Get BAM files for the specified condition
    bam_files = bam_dict[tissue][condition]
    
    # Check if all BAM files exist
    for bam_file in bam_files:
        if not os.path.exists(bam_file):
            print(f"Error: BAM file not found: {bam_file}")
            sys.exit(1)
            
    return bam_files

def calculate_mean_coverage(peak_file, tissue, condition):
    """
    Calculate mean coverage across replicates for all peaks in a condition.
    
    Parameters:
    - peak_file: path to BED file containing peaks
    - tissue: tissue type ('Neuron' or 'NSC')
    - condition: condition type ('Endo' or 'Exo')
    
    Returns:
    - dict: mapping of region coordinates to mean coverage
    """
    coverages = {}
    bam_files = get_sample_bams(tissue, condition)
    
    print(f"Processing {condition} peaks for {tissue} using {len(bam_files)} BAM files...")
    
    # Check if peak file exists
    if not os.path.exists(peak_file):
        print(f"Error: Peak file not found: {peak_file}")
        sys.exit(1)
    
    with open(peak_file) as f:
        for line in f:
            # Parse peak coordinates
            chrom, start, end = line.strip().split()[:3]
            start, end = int(start), int(end)
            region_key = f"{chrom}:{start}-{end}"
            
            # Calculate coverage for each replicate
            replicate_coverages = []
            for bam_file in bam_files:
                cov = calculate_coverage(bam_file, chrom, start, end)
                replicate_coverages.append(cov)
            
            # Store mean coverage across replicates
            coverages[region_key] = np.mean(replicate_coverages)
    
    return coverages

def process_peaks(tissue):
    """
    Process peaks for a given tissue type and generate summary statistics.
    
    Parameters:
    - tissue: tissue type ('Neuron' or 'NSC')
    """
    print(f"\nProcessing {tissue} peaks...")
    
    # Define input files
    endo_peaks = f"consensus_peaks/{tissue}_Endo_consensus.bed"
    exo_peaks = f"consensus_peaks/{tissue}_Exo_consensus.bed"
    
    # Check if consensus peak files exist
    for peak_file in [endo_peaks, exo_peaks]:
        if not os.path.exists(peak_file):
            print(f"Error: Consensus peak file not found: {peak_file}")
            sys.exit(1)
    
    # Calculate coverage for both conditions
    print("Calculating endogenous coverages...")
    endo_coverages = calculate_mean_coverage(endo_peaks, tissue, "Endo")
    print("Calculating exogenous coverages...")
    exo_coverages = calculate_mean_coverage(exo_peaks, tissue, "Exo")
    
    # Combine all unique regions from both conditions
    all_regions = set(endo_coverages.keys()) | set(exo_coverages.keys())
    
    # Process each region
    results = []
    for region in all_regions:
        chrom, pos = region.split(":")
        start, end = map(int, pos.split("-"))
        
        # Get mean coverage for each condition (default to 0 if no peak)
        endo_cov = endo_coverages.get(region, 0)
        exo_cov = exo_coverages.get(region, 0)
        
        # Calculate base mean (average coverage across conditions)
        base_mean = (endo_cov + exo_cov) / 2
        
        # Create entry for the region
        results.append({
            'gene': f"{chrom}:{start}-{end}",  # Using region as identifier
            'baseMean': round(base_mean, 2),
            'Endogenous_Promoter': region in endo_coverages,  # True if peak present in endogenous
            'Exogenous_Promoter': region in exo_coverages     # True if peak present in exogenous
        })
    
    # Convert to DataFrame and save
    df = pd.DataFrame(results)
    output_file = f"{tissue}_peak_analysis.csv"
    df.to_csv(output_file, index=False)
    
    # Print summary statistics
    print(f"\nResults for {tissue}:")
    print(f"Total regions analyzed: {len(df)}")
    print(f"Regions with Endogenous peaks: {df['Endogenous_Promoter'].sum()}")
    print(f"Regions with Exogenous peaks: {df['Exogenous_Promoter'].sum()}")
    print(f"Regions with both: {(df['Endogenous_Promoter'] & df['Exogenous_Promoter']).sum()}")
    print(f"Output saved to: {output_file}")

def main():
    """Main function to process both tissue types"""
    # Process each tissue type
    for tissue in ["Neuron", "NSC"]:
        process_peaks(tissue)

if __name__ == "__main__":
    main()


Processing Neuron peaks...
Calculating endogenous coverages...
Processing Endo peaks for Neuron using 2 BAM files...
Creating index for aligned/NeuM2.bam
Creating index for aligned/NeuM3.bam
Calculating exogenous coverages...
Processing Exo peaks for Neuron using 3 BAM files...
Creating index for aligned/NeuV1.bam
Creating index for aligned/NeuV2.bam
Creating index for aligned/NeuV3.bam
